In [ ]:
!pip install transformers
from transformers import pipeline
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
import numpy as np
import urllib.request
import csv

     |████████████████████████████████| 3.8 MB 8.4 MB/s 
     |████████████████████████████████| 6.5 MB 37.4 MB/s 
     |████████████████████████████████| 895 kB 55.6 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 53.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
classifier1 = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
classifier2 = pipeline("sentiment-analysis",model="ProsusAI/finbert")
classifier3 = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

#TWITTER ROBERTA
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)


Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [ ]:
#Read data
#post = pd.read_excel("SentimentalComparator.xlsx", index_col=0)

In [ ]:
#Create Dataset
data = {"Fact" : ["Fact prueba"], "Real": ["Real prueba"], "sadness-distilbert": ["sadness-distilbert prueba"], "joy-distilbert": ["joy-distilbert prueba"], "love-distilbert": ["love-distilbert prueba"], "love-distilbert": ["love-distilbert prueba"], "anger-distilbert": ["anger-distilbert prueba"], "fear-distilbert": ["fear-distilbert prueba"], "surprise-distilbert": ["surprise-distilbert prueba"], "finbert":["finbert prueba"], "roberta": ["roberta"], "positive-roberta":["positive-roberta prueba"], "neutral-roberta":["neutral-roberta prueba"], "negative-roberta":["negative-roberta prueba"] }
post = pd.DataFrame(data)
post.to_excel("SentimentalComparator.xlsx")

In [ ]:
#Read facts
data = pd.read_excel("/content/drive/MyDrive/Trabajo Fin de Grado/TFG/MongoDB/Reddit.xlsx", index_col=0)

In [ ]:
for x in range(len(data)):

  fact = data.loc[x, "RedditFact"]

  #DISTILBERT
  prediction = classifier1(fact, )
  for i in prediction:
    for x in i:
      #print(x["score"])
      if x["label"] == "sadness":
        sadness = x["score"]
      elif x["label"] == "joy":
        joy = x["score"]
      elif x["label"] == "love":
        love = x["score"]
      elif x["label"] == "anger":
        anger = x["score"]
      elif x["label"] == "fear":
        fear = x["score"]
      elif x["label"] == "surprise":
        surprise = x["score"]

  #FINBERT
  prediction = classifier2(fact)
  finbert = "{} : {}".format(prediction[0]["label"], prediction[0]["score"])

  #ROBERTA
  prediction = classifier3(fact)
  roberta = "{} : {}".format(prediction[0]["label"], prediction[0]["score"])

  #TWITTER ROBERTA
  encoded_input = tokenizer(fact, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  print(scores.shape[0])
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      if l == "positive":
        robertaPositive = "{}".format(np.round(float(s), 4))
      elif l == "neutral":
        robertaNeutral = "{}".format(np.round(float(s), 4))
      elif l == "negative":
        robertaNegative = "{}".format(np.round(float(s), 4))


  prueba = {"Fact":fact, "Real": 0, "sadness-distilbert": sadness, "joy-distilbert": joy, "love-distilbert": love, "anger-distilbert": anger, "fear-distilbert": fear, "surprise-distilbert": surprise, "finbert": finbert, "roberta": roberta, "positive-roberta": robertaPositive, "neutral-roberta": robertaNeutral, "negative-roberta": robertaNegative }
  li




In [ ]:
#SAVE
post.to_excel("SentimentalComparator.xlsx")